In [ ]:
#DAAE trained on celebA faces
import sys
sys.path.append('../')

from function import prep_data, make_new_folder, plot_losses, save_input_args, shift_x, plot_norm_losses
from dataload import CELEBA 
from models import DAE, DIS_Z, LINEAR_SVM

import torch
from torch import optim

from torchvision.utils import make_grid, save_image
from torchvision import transforms, datasets

import argparse

from time import time

import os
from os.path import join

import numpy as np

import matplotlib
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset

import pandas as pd

In [ ]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 100
        self.nz = 200
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = '../Experiments/DAAE1000/'
        self.commit = 'xxx'
        self.alpha = 1.0
        self.sigma = 0.1
        self.M = 5
        self.loss = 'BCE' #'BCE'
        self.loadDAE = False
        self.loadSVM = False
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 3
        self.multimodalZ = False


In [ ]:
# def get_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--root', default='/data', type=str) #data/datasets/LabelSwap
#     parser.add_argument('--batchSize', default=64, type=int)
#     parser.add_argument('--maxEpochs', default=10, type=int)
#     parser.add_argument('--nz', default=200, type=int)
#     parser.add_argument('--lr', default=1e-3, type=float)
#     parser.add_argument('--fSize', default=64, type=int)  # multiple of filters to use
#     parser.add_argument('--outDir', default='../../Experiments/DAAE1000/', type=str)
#     # parser.add_argument('--commit', required=True, type=str)
#     parser.add_argument('--commit', default='xxx', type=str)

#     parser.add_argument('--alpha', default=1.0, type=float)  # weight on the adversarial cost
#     parser.add_argument('--sigma', default=0.1, type=float)  # noise level
#     parser.add_argument('--M', default=5, type=int)  #number of sampling iterations
#     parser.add_argument('--loss', default='BCE', type=str) #'BCE' or 'MSE' currently supported
#     parser.add_argument('--loadDAE', action='store_true')
#     parser.add_argument('--loadSVM', action='store_true')
#     parser.add_argument('--load_DAE_from', default=None, type=str)
#     parser.add_argument('--evalMode', action='store_true')
#     parser.add_argument('--comment', type=str)
#     parser.add_argument('--momentum', default=0.9, type=float) 
#     parser.add_argument('--c', type=float, default=0.01) #for training the linearSVM for eval
#     parser.add_argument('--svmLR', type=float, default=1e-4)

#     parser.add_argument('--Ntest', default=100, type=int)
#     # parser.add_argument('--gpuNo', required=True, type=int)
#     parser.add_argument('--gpuNo', default=0, type=int)


#     return parser.parse_args()

def build_dis(dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = opts.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ

def svm_score(svm, y, x=None, enc=None, dae=None):
    '''
    EITHER
        take a data sample x AND a dae
    OR
        take an encoding
    and apply SVM and get score

    '''
    assert (x is not None) or (enc is not None)
    if enc is None:
        assert dae is not None
        enc = dae.encode(x)
    output = svm.forward(enc)
    score = svm.binary_class_score(output, y)
    return score

def eval_mode(dae, exDir, M, testLoader, svm=None):
    f = open(join(exDir, 'outputs.txt'), 'w')

    ## reconstruction error ##
    recError = []
    print('calculating reconstruction error...')
    for i, data in enumerate(testLoader):
        x, y = prep_data(data, useCUDA=dae.useCUDA)
        zTest = dae.encode(x)
        recTest = dae.decode(zTest)
        # recError.append(dae.rec_loss(recTest, x).data[0])
        recError.append(dae.rec_loss(recTest, x).detach().cpu().numpy())
    meanRecError = np.mean(recError)
    f.write('mean reconstruction error (non-corrupted): %0.5f' % (meanRecError))


    #eval samples ##TODO

    ## representation robustness (shift) ##
    print('performing robustness plot...')
    maxShift = x.size(2)//2
    maxShift_1 = x.size(3)//2
    step = 4
    axis = range(-maxShift, maxShift, step)
    axis_1 = range(-maxShift_1, maxShift_1, step)
    if x.size(2)%2 == 0:
        dim_x = maxShift*2//step
    else:
        dim_x = maxShift*2//step+1
    if x.size(3)%2 == 0:
        dim_y = maxShift_1*2//step
    else:
        dim_y = maxShift_1*2//step+1
    
    robustnessMap = torch.Tensor(dim_x, dim_y).fill_(0)
    classMap = torch.Tensor(dim_x, dim_y).fill_(0)
    x, y = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)  #take a batch of samples
    allShifts=[]
    enc00 = dae.encode(x)
    for j, dx in enumerate(axis):
        for i, dy in enumerate(axis_1):
            xShift = shift_x(x, dy, dx)
            encDxDy = dae.encode(xShift)
            # diff = [(torch.dot(encDxDy[k], enc00[k])/ (torch.norm(encDxDy[k])*torch.norm(enc00[k]))).data[0] for k in range(encDxDy.size(0))]
            # diff = [torch.dot(encDxDy[k], enc00[k]).data[0]/ ((torch.norm(encDxDy[k])*torch.norm(enc00[k])).data[0] + 1e-6) for k in range(encDxDy.size(0))]
            diff = [torch.dot(encDxDy[k], enc00[k]).detach().cpu().numpy()/ ((torch.norm(encDxDy[k])*torch.norm(enc00[k])).detach().cpu().numpy() + 1e-6) for k in range(encDxDy.size(0))]
            robustnessMap[j,i] = np.mean(diff)
            # classMap[j,i] = svm_score(svm, y, enc=encDxDy).data[0]
            classMap[j,i] = svm_score(svm, y, enc=encDxDy).detach().cpu()

            allShifts.append(xShift[0].cpu().data.numpy())

    print('saving images...')
    print(type(allShifts), np.shape(allShifts))
    save_image(torch.Tensor(np.asarray(allShifts)), join(exDir,'shiftImages.png'), nrow=16)
    print(robustnessMap)

    print('save maps as numpy array...')
    np.save(join(exDir, 'classMap.npy'), classMap.numpy())
    np.save(join(exDir, 'shiftMap.npy'), robustnessMap.numpy())

    # plot shift robustenss map
    fig0 = plt.figure()
    print(robustnessMap.min(), robustnessMap.max(), robustnessMap.size())
    f.write('\nrobustness min: %0.5f, max: %0.5f' % (robustnessMap.min(), robustnessMap.max()))
    plt.imshow(robustnessMap.numpy(), extent=[-maxShift, maxShift, -maxShift, maxShift], vmin=0.0, vmax=1.0)
    plt.xlabel('DX')
    plt.ylabel('DY')
    plt.title('Robustness to shifts in x and y')
    plt.colorbar()
    plt.savefig(join(exDir, 'ShiftRobustness.png'))

    #Plot shift robusteness classification map
    fig1 = plt.figure()
    f.write('\nshift robustenss accuracy min: %0.5f, max: %0.5f' % (classMap.min(), classMap.max()))
    f.write('\nAccuray Volume (sum of elements in accuracy shift map): %0.5f' % (np.clip(classMap.numpy(), 0.5, 1).sum()))
    plt.imshow(classMap.numpy(), extent=[-maxShift, maxShift, -maxShift, maxShift], vmin=0.5, vmax=1.0)
    plt.xlabel('DX')
    plt.ylabel('DY')
    plt.title('Classiciation Robustness to shifts in x and y')
    plt.colorbar()
    plt.savefig(join(exDir, 'ClassificationShiftRobustness.png'))

    ## Compare histograms for enc, z_samples and encCorr
    fig2 = plt.figure()
    # nEnc, bEnc, _ = plt.hist(enc00.cpu().data.numpy().flatten(), 100, normed=True)
    nEnc, bEnc, _ = plt.hist(enc00.cpu().data.numpy().flatten(), 100, density=True, stacked=True)
    xcorr = dae.corrupt(x)
    encCorr = dae.encode(xcorr)
    # nEncCorr, bEncCorr, _ = plt.hist(encCorr.cpu().data.numpy().flatten(), 100, normed=True)
    # nNorm, bNorm, _ = plt.hist(dae.sample_z(10000).cpu().data.numpy().flatten(), 100, normed=True)
    nEncCorr, bEncCorr, _ = plt.hist(encCorr.cpu().data.numpy().flatten(), 100, density=True, stacked=True)
    nNorm, bNorm, _ = plt.hist(dae.sample_z(10000).cpu().data.numpy().flatten(), 100, density=True, stacked=True)
    fig3 = plt.figure()
    plt.plot(bEnc[1:], nEnc, label='encoding')
    plt.plot(bEncCorr[1:], nEncCorr, label='corrupted encoding')
    plt.plot(bNorm[1:], nNorm, label='Normal')
    plt.title('Comparing Encodings')
    plt.xlabel('Value')
    plt.ylabel('pdf')
    plt.legend()
    plt.savefig(join(exDir, 'HisEnc.png'))

    #save all histograms:
    np.save(join(exDir, 'HistEnc.npy'), [nEnc,bEnc])
    np.save(join(exDir), 'HistEncCorr.npy', [nEncCorr, bEncCorr])
    np.save(join(exDir), 'prior.npy', [nNorm, bNorm])

    #sampling
    print('sampling...')
    sampleDir = join(exDir,'FinalSamples')
    try:
        os.mkdir(sampleDir)
    except OSError: print('file alread exists')
    dae.sample_x(opts.M, sampleDir)

    ## classification test score
    if svm is not None:
        'Do classification'
        testScore = 0
        for i, data in enumerate(testLoader):
            x, y = prep_data(data, useCUDA=svm.useCUDA)
            score = svm_score(svm, y, x=x, dae=dae) 
            testScore+=score
    testScore /= (i+1)
    # f.write('\nSVM classification (test) score:'+str(testScore.data[0]))
    f.write('\nSVM classification (test) score:'+str(testScore.detach().cpu().numpy()))
    f.close()




def train_svm(dae, svm, trainLoader, testLoader, exDir, lr):
    '''
    Data y is [0,1]
    For training SVM must be -1, 1
    To eval data put back to [0,1]

    To get loss use [-1,1] for train and test
    To get score use [0,1] for train and test
    '''
    print('training svm...')
    dae.eval()
    optimSVM = optim.SGD(svm.parameters(), lr=lr) #optimizer  

    f = open(join(exDir, 'svmOpts.txt'), 'w')
    f.write('smvLR: %0.5f\nc: %0.5f\n' % (lr, svm.c))
    f.close() 


    svmLoss = {'train':[], 'test':[]}
    for epoch in range(opts.maxEpochs):
        epochLoss_svm=0
        svm.train()
        T = time()
        for i, data in enumerate(trainLoader):
            x, y = prep_data(data, useCUDA=svm.useCUDA)  #prep data as a var
            inputs = dae.encode(x)  #get encodings as input
            output = svm.forward(inputs)  #get output
            loss = svm.loss(output, y * 2 - 1)  #calc loss 
            optimSVM.zero_grad()  #zero grad
            loss.backward()  #backwards
            optimSVM.step()  #step
            # epochLoss_svm+=loss.data[0]
            epochLoss_svm+=loss.detach().cpu().numpy()
            if i%100 == 0:
                print('[%d, %i] loss: %0.5f, time: %0.3f' % (epoch, i, epochLoss_svm/(i+1), time() - T))
        svm.save_params(exDir)
        svmLoss['train'].append(epochLoss_svm/(i+1))

        #test loss:
        svm.eval()
        xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=svm.useCUDA)
        testInputs = dae.encode(xTest)
        testOutputs = svm.forward(testInputs)
        testLoss = svm.loss(testOutputs, yTest * 2 - 1)
        # svmLoss['test'].append(testLoss.data[0])
        svmLoss['test'].append(testLoss.detach().cpu().numpy())

        if epoch > 1:
            plot_losses(svmLoss, exDir=exDir, epochs=epoch+1, title='SVM_loss')

        #Do classification
        testScore = svm.binary_class_score(testOutputs, yTest) #has threshold as zero for testOutputs in [-1,1]
        trainScore = svm.binary_class_score(output, y) #has threshold as zero for output in [-1,1]
        f = open(join(exDir, 'svm.txt'), 'w')
        # f.write('trainScore: %f \ntestScore: %f ' \
        #  % (trainScore.mean().data[0], testScore.mean().data[0]))
        f.write('trainScore: %f \ntestScore: %f ' \
         % (trainScore.mean().detach().cpu().numpy(), testScore.mean().detach().cpu().numpy()))
        f.close()

    return svm

In [ ]:
window_len = 512 # 512
stride_len = 20 # 100
# act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
act_list = [1, 2]


In [ ]:
# opts = get_args()
X=[]
user_labels=[]
act_labels=[]

# columns for IMU data
imu_locs = [4,5,6, 10,11,12, 13,14,15, 
            21,22,23, 27,28,29, 30,31,32, 
            38,39,40, 44,45,46, 47,48,49
           ] 

scaler = MinMaxScaler()
# scaler = StandardScaler()

for uid in np.arange(1,10):
    path = '../../../PAMAP2_Dataset/Protocol/subject10' + str(uid) + '.dat'
    df = pd.read_table(path, sep=' ', header=None)
    act_imu_filter = df.iloc[:, imu_locs] 


    for act_id in act_list:
        act_filter =  act_imu_filter[df.iloc[:, 1] == act_id]
        act_data = act_filter.to_numpy()
        if act_data.shape[0] > 0:      
            # scaler = StandardScaler()
            # scaler = MinMaxScaler()
            if uid==1 and act_id == act_list[0]:
                scaler.fit(act_data)
            act_data = scaler.transform(act_data)
            
        act_data = np.transpose(act_data)

        # sliding window segmentation
        start_idx = 0
        while start_idx + window_len < act_data.shape[1]:
            window_data = act_data[:, start_idx:start_idx+window_len]
            downsamp_data = window_data[:, ::3] # downsample from 100hz to 33.3hz
            downsamp_data = np.nan_to_num(downsamp_data) # remove nan

            X.append(downsamp_data)
            user_labels.append(uid)
            act_labels.append(act_id)
            start_idx = start_idx + stride_len


In [ ]:
X = np.array(X).astype('float32')
X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) # convert list to numpy array
act_labels = np.array(act_labels).astype('float32')
act_labels = act_labels.reshape(act_labels.shape[0],1)

In [ ]:
print(X.shape)
print(act_labels.shape)
# plt.imshow(X[1])
# plt.show()
# plt.pause(1e-6)


In [ ]:
# if __name__=='__main__':
# opts = get_args()
opts = Args()

# #Load data
# print('Prepare data loaders...')
# transform = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip()])
# trainDataset = CELEBA(root=opts.root, train=True, transform=transforms.ToTensor())
# trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=opts.batchSize, shuffle=True)

# testDataset = CELEBA(root=opts.root, train=False, transform=transforms.ToTensor())
# testLoader = torch.utils.data.DataLoader(testDataset, batch_size=opts.batchSize, shuffle=False)
# print('Data loaders ready.')

print('Prepare data loaders...')

dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(act_labels))

# Train/Test dataset split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainDataset, testDataset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainLoader = torch.utils.data.DataLoader(trainDataset,
    batch_size=opts.batchSize, shuffle=True) 

testLoader = torch.utils.data.DataLoader(testDataset,
    batch_size=opts.batchSize, shuffle=False)
print('Data loaders ready.')


In [ ]:
print(torch.std(trainDataset[0][0]))

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.functional import binary_cross_entropy as bce
# from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm
from torch import nn

In [ ]:
class DAE(nn.Module):

    def __init__(self, nz, imSize, fSize=2, sigma=0.1, multimodalZ=False):  #sigma is the corruption level
        super(DAE, self).__init__()
        #define layers here

        self.fSize = fSize
        self.nz = nz
        self.imSize = imSize
        self.sigma = sigma
        self.multimodalZ = multimodalZ

        inSize = int(imSize / ( 2 ** 4))
        self.inSize = inSize

        if multimodalZ:
            NZ = 2
        else:
            NZ = nz
        self.enc1 = nn.Conv2d(1, fSize, 5, stride=2, padding=2)
        # self.enc1 = nn.Conv2d(1, fSize, (X.shape[2], X.shape[3]), stride=2, padding=2)
        # self.enc1 = nn.Conv2d(3, fSize, 5, stride=2, padding=2)
        self.enc2 = nn.Conv2d(fSize, fSize * 2, 5, stride=2, padding=2)
        self.enc3 = nn.Conv2d(fSize * 2, fSize * 4, 5, stride=2, padding=2)
        self.enc4 = nn.Conv2d(fSize * 4, fSize * 8, 5, stride=2, padding=2)
        # self.enc5 = nn.Linear((fSize * 8) * inSize * inSize, NZ)
        self.enc5 = nn.Linear(11264, NZ)
        
        # self.dec1 = nn.Linear(NZ, (fSize * 8) * inSize * inSize)
        self.dec1 = nn.Linear(NZ, 11264)
        self.dec2 = nn.ConvTranspose2d(fSize * 8, fSize * 4, 3, stride=2, padding=1, output_padding=1)
        self.dec3 = nn.ConvTranspose2d(fSize * 4, fSize * 2, 2, stride=2, padding=1, output_padding=1)
        self.dec4 = nn.ConvTranspose2d(fSize * 2, fSize, 3, stride=2, padding=1, output_padding=1)
        # self.dec5 = nn.ConvTranspose2d(fSize, 3, 3, stride=2, padding=1, output_padding=1)
        self.dec5 = nn.ConvTranspose2d(fSize, 1, 2, stride=2, padding=1, output_padding=1)

        self.relu = nn.ReLU()

        self.useCUDA = torch.cuda.is_available()

    def norm_prior(self, noSamples=25):
        z = torch.randn(noSamples, self.nz)
        return z

    def multi_prior(self, noSamples=25, mode=None):
        #make a 2D sqrt(nz)-by-sqrt(nz) grid of gaussians
        num = np.sqrt(self.nz) #no of modes in x and y
        STD = 1.0
        modes = np.arange(-num,num)
        p = np.random.uniform(0, num,(noSamples*2))

        if mode is None:
            mu = modes[np.floor(2 * p).astype(int)]
        else:
            mu = modes[np.ones((noSamples, 2), dtype=int) * int(mode)]

        z = torch.Tensor(mu).view(-1,2) + STD * torch.randn(noSamples, 2)
        return z

    def encode(self, x):
        #define the encoder here return mu(x) and sigma(x)
        x = self.relu(self.enc1(x))
        x = self.relu(self.enc2(x))
        x = self.relu(self.enc3(x))
        x = self.relu(self.enc4(x))
        x = x.view(x.size(0), -1)
        x = self.enc5(x)

        return x

    # def corrupt(self, x):
    #     noise = self.sigma * torch.randn(x.size()).type_as(x)
    #     return x + noise
    
    def corrupt(self, x):
        num_zeros = int(torch.numel(x) * self.sigma)
        if self.useCUDA:
            mask = torch.ones(torch.numel(x)).cuda()
        else:
            mask = torch.ones(torch.numel(x))
        mask[:num_zeros] = 0
        mask = mask[torch.randperm(mask.shape[0])]
        mask = mask.reshape(x.shape)#
        return torch.mul(x, mask)
         
    def sample_z(self, noSamples=25, mode=None):
        if not self.multimodalZ:
            z = self.norm_prior(noSamples=noSamples)
        else:
            z = self.multi_prior(noSamples=noSamples, mode=mode)
        if self.useCUDA:
            return z.cuda()
        else:
            return z

    def decode(self, z):
        #define the decoder here
        # print('z')
        z = self.relu(self.dec1(z))
        # z = z.view(z.size(0), -1, self.inSize, self.inSize)
        z = z.view(z.size(0), -1, 2, 11)
        z = self.relu(self.dec2(z))
        z = self.relu(self.dec3(z))
        z = self.relu(self.dec4(z))
        z = torch.sigmoid(self.dec5(z))

        return z

    def forward(self, x):
        # the outputs needed for training
        x_corr = self.corrupt(x)
        z = self.encode(x_corr)
        return z, self.decode(z)

    def rec_loss(self, rec_x, x, loss='BCE'):
        if loss == 'BCE':
            return torch.mean(bce(rec_x, x, size_average=True))  #not averaged over mini-batch if size_average=FALSE and is averaged if =True 
        elif loss == 'MSE':
            return torch.mean(F.mse_loss(rec_x, x, size_average=True))
        else:
            print('unknown loss:'+loss)

    def save_params(self, exDir):
        print('saving params...')
        torch.save(self.state_dict(), join(exDir, 'dae_params'))

    def load_params(self, exDir):
        print('loading params...')
        self.load_state_dict(torch.load(join(exDir, 'dae_params')))

    def sample_x(self, M, exDir, z=None):
        if z == None:
            z = self.sample_z(noSamples=25)
        if not self.multimodalZ:
            x_i = self.decode(z)
            save_image(x_i.data, join(exDir, 'samples0.png'))
            for i in range(M):
                z_i, x_i = self.forward(x_i) #corruption already in there!
                save_image(x_i.data, join(exDir, 'samples'+str(i+1)+'.png'))
        else:
            #show samples from a few modes
            maxModes = min(self.nz, 5)  #show at most 5 modes
            for mode in range(maxModes):
                z = self.sample_z(noSamples=25, mode=mode)
                x_i = self.decode(z)
                save_image(x_i.data, join(exDir, 'mode'+str(mode)+'_samples.png'))
                for i in range(M):
                    z_i, x_i = self.forward(x_i) #corruption already in there!
                    save_image(x_i.data, join(exDir, 'mode'+str(mode)+'_samples'+str(i+1)+'.png'))

In [ ]:
#Create model
dae = DAE(nz=opts.nz, imSize=64, fSize=opts.fSize, sigma=opts.sigma, multimodalZ=opts.multimodalZ) #sigma=level of corruption
dis, NZ = build_dis(dae=dae, multimodalZ=opts.multimodalZ)
svm = LINEAR_SVM(nz=NZ, c=opts.c) #model

if dae.useCUDA:
    torch.cuda.set_device(opts.gpuNo)
    print('using gpu:', torch.cuda.current_device())
    dae.cuda()
    dis.cuda()
    svm.cuda()

In [ ]:
if opts.loadDAE:  #should load DAE if in eval mode
    print('loading DAE...')
    dae.load_params(opts.load_DAE_from)

if opts.loadSVM:
        svm.load_params(opts.load_DAE_from) #use SVM @ same location as DAE [may not be one there]

if opts.evalMode & (not opts.loadSVM):  #to train an SVM for eval
        svm = train_svm(dae=dae, svm=svm, trainLoader=trainLoader, testLoader=testLoader, exDir=opts.load_DAE_from, lr=opts.svmLR)

if opts.evalMode:
    assert opts.loadDAE == True
    eval_mode(dae, opts.load_DAE_from, opts.M, testLoader, svm=svm)
    exit()
else:
    #Create a folder for this experiment
    exDir = make_new_folder(opts.outDir)
    print('Outputs will be saved to:',exDir)
    save_input_args(exDir, opts)  #save training opts

In [ ]:
#Create optimizers
optimDAE = optim.RMSprop(dae.parameters(), lr = opts.lr)
optimDIS = optim.RMSprop(dis.parameters(), lr = opts.lr, momentum=opts.momentum)

#Keeping track of training
losses = {'enc': [], 'rec': [], 'dis':[], 'test rec':[]}

In [ ]:
# xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)


In [ ]:
# noise = torch.round(torch.randn(xTest.size()).type_as(xTest))

In [ ]:
# noise = torch.randint(0, 2, xTest.size()).type_as(xTest)

In [ ]:
# print(xTest)

In [ ]:
# save_image(dae.corrupt(xTest.data), join(exDir, 'corrupted.png'))

In [ ]:
# torch.min(xTest[0])

In [ ]:
#Start training
with torch.autograd.set_detect_anomaly(True):
    for e in range(opts.maxEpochs):

        epochEncLoss=0
        epochRecLoss=0
        epochDisLoss=0

        for i, data in enumerate(trainLoader):
            # print(i)

            T = time()

            dae.train()
            dis.train()

            x, y = prep_data(data, useCUDA=dae.useCUDA)
            # print(x.shape)
            
            # get outputs
            zFake, xRec = dae.forward(x)

            # clac losses
            recLoss = dae.rec_loss(xRec, x, loss=opts.loss)  #loss='BCE' or 'MSE'
            
            optimDAE.zero_grad()
            recLoss.backward()
            optimDAE.step()            
            
            
            zFake, xRec = dae.forward(x)    
            
            disLoss = dis.dis_loss(zFake)


            
            #do updates

            optimDIS.zero_grad()
            disLoss.backward()
            optimDIS.step()
            
            encLoss = dis.gen_loss(zFake)
            
            optimDAE.zero_grad()
            encLoss.backward()
            optimDAE.step()                   
                        
            # encLoss = dis.gen_loss(zFake)
            # daeLoss = recLoss + opts.alpha * encLoss

            # print(recLoss)
            # print(encLoss)
            # print(disLoss)            
            

            
            # print(encLoss.data)

            # storing losses for plotting later
            # epochEncLoss+=encLoss.data[0]
            # epochRecLoss+=recLoss.data[0]
            # epochDisLoss+=disLoss.data[0]
            epochEncLoss+=encLoss.detach().cpu().numpy()
            epochRecLoss+=recLoss.detach().cpu().numpy()
            epochDisLoss+=disLoss.detach().cpu().numpy()
            
            if i%100 == 0:
                # print('[%d, %d] enc: %0.5f, rec: %0.5f, dis: %0.5f, time: %0.3f' % (e, i, encLoss.data[0], recLoss.data[0], disLoss.data[0], time() - T))
                print('[%d, %d] enc: %0.5f, rec: %0.5f, dis: %0.5f, time: %0.3f' % (e, i, encLoss.detach().cpu().numpy(), recLoss.detach().cpu().numpy(), disLoss.detach().cpu().numpy(), time() - T))

        # storing losses for plotting later
        losses['enc'].append(epochEncLoss/i)
        losses['rec'].append(epochRecLoss/i)
        losses['dis'].append(epochDisLoss/i)

        #### Test
        dae.eval()
        dis.eval()

        #get test outuputs and losses
        xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)
        zTest, recTest = dae.forward(xTest)  #N.B. corruption in here
        recLossTest = dae.rec_loss(recTest, xTest)

        #Plot losses
        # losses['test rec'].append(recLossTest.data[0])
        losses['test rec'].append(recLossTest.detach().cpu().numpy())

        if e > 0: #only one point for test rec otherwise
            plot_losses(losses, exDir, epochs=e+1)
            plot_norm_losses(losses, exDir)

        #save parameters
        dae.save_params(exDir)
        dis.save_params(exDir)

        #Save images of original and rec
        save_image(xTest.data, join(exDir, 'original.png'))
        save_image(recTest.data, join(exDir, 'rec.png'))

        #Save samples
        sampleDir = join(exDir,'epoch_'+str(e))
        os.mkdir(sampleDir)
        print('sample dir:', sampleDir)
        dae.sample_x(opts.M, sampleDir)



In [ ]:
if not opts.evalMode:
    eval_mode(dae=dae, exDir=exDir, M=20, testLoader=testLoader, svm=svm)
    svm = train_svm(dae=dae, svm=svm, trainLoader=trainLoader, testLoader=testLoader, exDir=exDir, lr=opts.svmLR)


#Train a linear-SVM classifier on the enocdings